In [2]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [3]:
PATH = Path('../input/optiver-realized-volatility-prediction')


In [4]:
df_train = pd.read_csv(PATH/'train_with_ftrs.csv').drop('row_id', axis=1)

In [5]:
df_train

,target,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,...,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,stock_id,stock_id_target_enc
0,0.004136,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,...,0.002006,40.0,3179.0,2.750000,0.001308,21.0,1587.0,2.571429,0,0.004041
1,0.001445,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,...,0.000901,30.0,1289.0,1.900000,0.000587,16.0,900.0,2.250000,0,0.004039
2,0.002168,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,...,0.001961,25.0,2161.0,2.720000,0.001137,12.0,1189.0,3.166667,0,0.004041
3,0.002195,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,...,0.001561,15.0,1962.0,3.933333,0.001089,9.0,1556.0,5.111111,0,0.004013
4,0.001747,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,...,0.000871,22.0,1791.0,4.045455,0.000453,11.0,1219.0,4.909091,0,0.004032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,0.003461,0.003691,0.005876,0.000361,0.000878,0.000091,-0.000202,161.638710,406.045161,0.999582,...,0.002171,37.0,2570.0,2.783784,0.001451,18.0,796.0,2.055556,126,0.005312
428928,0.003113,0.004104,0.004991,0.000295,0.000706,0.000126,-0.000142,150.578475,243.322870,1.002476,...,0.002180,43.0,2323.0,3.418605,0.001791,20.0,1107.0,3.550000,126,0.005327
428929,0.004070,0.003118,0.006019,0.000394,0.000739,0.000189,-0.000192,254.406250,348.093750,1.001082,...,0.001921,35.0,3740.0,2.800000,0.001580,24.0,2750.0,2.541667,126,0.005321
428930,0.003357,0.003661,0.005362,0.000231,0.000530,0.000143,-0.000134,145.654135,426.416040,1.001809,...,0.002051,80.0,9389.0,2.925000,0.001520,43.0,5150.0,2.813953,126,0.005328


In [6]:
def rmspe_np(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

In [7]:
X = df_train.drop(['target'],axis=1)
y = df_train['target']

In [8]:
kf = KFold(n_splits=5, random_state=19901028, shuffle=True)
trn_idx, val_idx = first(kf.split(X))

# Random Forest baseline

In [9]:

X_train, y_train = X.loc[trn_idx], y[trn_idx]
X_valid, y_valid = X.loc[val_idx], y[val_idx]
weights = 1/np.square(y_train)
    
model = RandomForestRegressor(min_samples_leaf=60,max_features='log2', n_jobs=-1)
model.fit(X_train, y_train, weights)

RandomForestRegressor(max_features='log2', min_samples_leaf=60, n_jobs=-1)

In [10]:
y_pred = model.predict(X_valid)
rmspe_np(y_true = y_valid, y_pred = y_pred)

0.23406604052493715

# Neural Net

In [11]:
df_train.stock_id = df_train.stock_id.astype('category')

In [12]:
df_train = df_train.drop('stock_id_target_enc', axis=1)

In [13]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [14]:
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                      splits=[list(trn_idx), list(val_idx)], y_names='target')

In [15]:
dls = to_nn.dataloaders(2048)

In [16]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [17]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [24]:
learn = tabular_learner(dls, y_range=(0,.1), layers=[400,200],
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe))

In [ ]:
learn.fit_one_cycle(2000, 1e-2)

epoch,train_loss,valid_loss,rmspe,time
0,5.183312,3.920279,3.982538,00:03
1,2.438336,1.637620,1.686114,00:03
2,1.074984,0.668816,0.685203,00:03
3,0.448996,0.345495,0.348772,00:03
4,0.315515,0.277901,0.283227,00:03
5,0.283283,0.325084,0.327942,00:03
6,0.258533,0.256339,0.258968,00:03
7,0.246724,0.247983,0.250847,00:03
8,0.246558,0.247228,0.250428,00:03
9,0.244810,0.262337,0.266258,00:03


In [100]:
preds, tars = learn.get_preds()

In [101]:
preds.shape

torch.Size([85787, 1])

In [102]:
preds_nn = preds.view(-1).numpy()

In [103]:
rmspe_np(y_true = y_valid, y_pred = preds_nn)

0.22675025973279012

In [104]:
preds_ens = (y_pred + preds_nn)/2

In [105]:
rmspe_np(y_true = y_valid, y_pred = preds_ens)

0.22713026508344736

In [106]:
learn.save('tabular226.pth')

Path('models/tabular226.pth.pth')